In [43]:
from nltk.corpus import wordnet,stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
import string
import nltk
from sklearn.model_selection import GridSearchCV,cross_val_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [44]:
training_data = pd.read_csv('Datasets/Train.csv')
test_text = pd.read_csv('Datasets/Test.csv')['text']

In [45]:
texts = training_data['text']
Y_train = training_data['airline_sentiment']

In [46]:
X =[]
X_test = []
for text in texts:
    X.append(word_tokenize(text))

for text in test_text:
    X_test.append(word_tokenize(text))
    

In [47]:
stopwords = stopwords.words('english') + list(string.punctuation)
lemmatizer = WordNetLemmatizer()
def clean_doc(doc):
    clean_words=[]
    for word in doc:
        if word.lower() not in stopwords and ( word.isalpha()):
            clean_words.append( lemmatizer.lemmatize( word , simple(pos_tag([word])[0][1])).lower())
    return clean_words

In [48]:
def simple(tag):
    if tag.startswith('N'):
        return wordnet.NOUN
    if tag.startswith('V'):
        return wordnet.VERB
    if tag.startswith('A'):
        return wordnet.ADV
    if tag.startswith('J'):
        return wordnet.ADJ
    else:
        return wordnet.NOUN

In [49]:
X_train= [ ' '.join( clean_doc(doc)) for doc in X ]
X_test= [ ' '.join( clean_doc(doc)) for doc in X_test ]
X_test[0]

'americanair car gng dfw pulled ago icy road aa since ca reach arpt wat'

In [50]:
Y_train = Y_train.replace({
    'negative' : 0,
    'neutral' : 1,
    'positive' : 2,
})
Y_train

0        0
1        2
2        2
3        0
4        0
        ..
10975    1
10976    2
10977    0
10978    0
10979    0
Name: airline_sentiment, Length: 10980, dtype: int64

In [51]:
cv = CountVectorizer(max_features=3000)
cv.fit(X_train)

CountVectorizer(max_features=3000)

In [52]:
cv.get_feature_names()

['aa',
 'aadvantage',
 'aafail',
 'abandon',
 'abc',
 'ability',
 'able',
 'aboard',
 'abq',
 'absolute',
 'absolutely',
 'absurd',
 'abt',
 'abuse',
 'abysmal',
 'ac',
 'accept',
 'acceptable',
 'accepted',
 'access',
 'accident',
 'accidentally',
 'accommodate',
 'accommodation',
 'accomplish',
 'accord',
 'accordingly',
 'account',
 'accountability',
 'acct',
 'accurate',
 'acknowledge',
 'across',
 'act',
 'action',
 'actual',
 'actually',
 'ad',
 'add',
 'additional',
 'address',
 'adds',
 'addtl',
 'admiral',
 'admirals',
 'admit',
 'advance',
 'advantage',
 'advertise',
 'advice',
 'advise',
 'advisory',
 'affect',
 'affected',
 'afford',
 'affordable',
 'afternoon',
 'age',
 'agency',
 'agent',
 'agents',
 'aggressive',
 'ago',
 'agree',
 'agt',
 'ah',
 'ahead',
 'ahhhh',
 'ahold',
 'ai',
 'air',
 'airbus',
 'aircanada',
 'aircraft',
 'airfare',
 'airline',
 'airlinegeeks',
 'airlines',
 'airplane',
 'airport',
 'airway',
 'airways',
 'aisle',
 'alaska',
 'albany',
 'alert',
 '

In [53]:
X_train = cv.transform(X_train)
X_test = cv.transform(X_test)

In [54]:
X_train = X_train.todense()
X_test = X_test.todense()

In [55]:
from sklearn.naive_bayes import MultinomialNB
model1 = MultinomialNB()
cross_val_score(model1,X_train,Y_train).mean()

0.7636612021857924

In [56]:
from sklearn.linear_model import LogisticRegression
model2 = LogisticRegression(max_iter=2000,multi_class='ovr')
cross_val_score(model2,X_train,Y_train).mean()

0.7863387978142077

In [57]:
from sklearn.ensemble import RandomForestClassifier
model3 = RandomForestClassifier()
cross_val_score(model3,X_train,Y_train).mean()

0.7544626593806921

In [58]:
# from sklearn.svm import SVC
# model4 = SVC()
# cross_val_score(model4,X_train,Y_train).mean()

In [59]:
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
model = Sequential()
model.add(Dense(1024,activation='relu',input_dim = 3000))
model.add(Dense(512,activation='sigmoid'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='sigmoid'))
model.add(Dense(64,activation='relu'))
model.add(Dense(3,activation='sigmoid'))

In [60]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [61]:
model.fit(X_train,  to_categorical(Y_train)  ,epochs=20,batch_size = 50)

Epoch 1/20
220/220 [==============================] - 6s 28ms/step - loss: 0.6751 - accuracy: 0.7233
Epoch 2/20
220/220 [==============================] - 6s 29ms/step - loss: 0.4201 - accuracy: 0.8420
Epoch 3/20
220/220 [==============================] - 6s 27ms/step - loss: 0.2767 - accuracy: 0.9042
Epoch 4/20
220/220 [==============================] - 6s 27ms/step - loss: 0.1490 - accuracy: 0.9524
Epoch 5/20
220/220 [==============================] - 6s 26ms/step - loss: 0.0796 - accuracy: 0.9760
Epoch 6/20
220/220 [==============================] - 6s 26ms/step - loss: 0.0540 - accuracy: 0.9838
Epoch 7/20
220/220 [==============================] - 6s 26ms/step - loss: 0.0398 - accuracy: 0.9887
Epoch 8/20
220/220 [==============================] - 6s 27ms/step - loss: 0.0325 - accuracy: 0.9897
Epoch 9/20
220/220 [==============================] - 6s 26ms/step - loss: 0.0352 - accuracy: 0.9889
Epoch 10/20
220/220 [==============================] - 6s 26ms/step - loss: 0.0305 - accura

In [75]:
# 74%
# predict = model.predict_classes(X_test)
model2 = LogisticRegression(max_iter=2000,multi_class='ovr')
model2.fit(X_train,Y_train)
predict  = model2.predict(X_test)
predict

array([0, 0, 0, ..., 0, 2, 1])

In [76]:
predict = pd.Series(predict).replace({
    0:'negative',
    1:'neutral',
    2:'positive'
})
predict

0       negative
1       negative
2       negative
3       negative
4       positive
          ...   
3655    negative
3656    negative
3657    negative
3658    positive
3659     neutral
Length: 3660, dtype: object

In [77]:
predict.to_csv('Prediction.csv',index=False,header=False)

# Accuracy : 78.33 %